In [ ]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt


In [ ]:

monday_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv"
tuesday_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Tuesday-WorkingHours.pcap_ISCX.csv"
wednesday_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Wednesday-workingHours.pcap_ISCX.csv"
thursday_m_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
thursday_a_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
friday_dos_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
friday_portscan_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"
friday_morning_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv"

monday = pd.read_csv(monday_path)
tuesday = pd.read_csv(tuesday_path)
wednesday = pd.read_csv(wednesday_path)
thursday_m = pd.read_csv(thursday_m_path)
thursday_a = pd.read_csv(thursday_a_path)
friday_dos = pd.read_csv(friday_dos_path)
friday_portscan = pd.read_csv(friday_portscan_path)
friday_morning = pd.read_csv(friday_morning_path)
full_df = pd.concat([monday, tuesday, wednesday, thursday_m, thursday_a, 
                  friday_dos, friday_portscan, friday_morning], ignore_index=True)

print("Dataset shape:", full_df.shape)
print("Columns:", full_df.columns.tolist())


Dataset shape: (2830743, 79)
Columns: [' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag 

In [ ]:

# Encode labels (BENIGN = 0, ATTACK = 1)
def encode_labels(df):
    le = LabelEncoder()
    df[' Label'] = le.fit_transform(df[' Label'])
    df[' Label'] = df[' Label'].apply(
        lambda x: 0 if le.classes_[x] == 'BENIGN' else 1
    )
    return df

full_df = encode_labels(full_df)

# Keep only numeric columns
full_df = full_df.select_dtypes(include=[np.number])
print("Dataset after label encoding + numeric filter:", full_df.shape)

# Features & Labels + Cleaning
X = full_df.drop(' Label', axis=1).values
y = full_df[' Label'].values

# Replace inf → nan
X[np.isinf(X)] = np.nan

# Drop rows with NaN
mask = ~np.isnan(X).any(axis=1)
X, y = X[mask], y[mask]

print("Final dataset shape:", X.shape, "Labels shape:", y.shape)

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train:", X_train.shape, "Test:", y_test.shape)


#  Config for Federated run

NUM_CLIENTS = 5
NUM_EPOCHS = 5           # federated rounds
LOCAL_EPOCHS = 1         # local epochs per round
BATCH_SIZE = 64
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

FED_NONIID = True        # False = IID split, True = Non-IID split
USE_BOOTSTRAP = True     # Toggle bootstrapping for client sampling
BOOTSTRAP_RATIO = 0.8    # Resample % of each client dataset

#  Partitioning Functions
def iid_partition(X, y, num_clients=NUM_CLIENTS, seed=42):
    rng = np.random.default_rng(seed)
    idx = np.arange(len(X))
    rng.shuffle(idx)
    splits = np.array_split(idx, num_clients)
    return [(X[s], y[s]) for s in splits]

def noniid_partition(X, y, num_clients=NUM_CLIENTS, seed=42):
    np.random.seed(seed)
    idx_0 = np.where(y == 0)[0]   # BENIGN
    idx_1 = np.where(y == 1)[0]   # ATTACK
    np.random.shuffle(idx_0); np.random.shuffle(idx_1)

    size_0 = len(idx_0) // num_clients
    size_1 = len(idx_1) // num_clients

    clients_data = []
    for i in range(num_clients):
        if i % 2 == 0:  # even clients → mostly BENIGN
            main_idx = idx_0[size_0*i : size_0*(i+1)]
            other_idx_end = min(len(idx_1), size_1*(i//3 + 1))
            other_idx = idx_1[size_1*i//3 : other_idx_end]
        else:           # odd clients → mostly ATTACK
            main_idx = idx_1[size_1*i : size_1*(i+1)]
            other_idx_end = min(len(idx_0), size_0*(i//3 + 1))
            other_idx = idx_0[size_0*i//3 : other_idx_end]

        indices = np.concatenate((main_idx, other_idx))
        np.random.shuffle(indices)
        clients_data.append((X[indices], y[indices]))
    return clients_data

# Bootstrapping Function

def bootstrap_client_data(clients_data, ratio=BOOTSTRAP_RATIO, seed=42):
    rng = np.random.default_rng(seed)
    boot_clients = []
    for Xc, yc in clients_data:
        n_samples = int(len(Xc) * ratio)
        idx = rng.choice(len(Xc), n_samples, replace=True)
        boot_clients.append((Xc[idx], yc[idx]))
    return boot_clients

#  Get Clients Data
clients_data = (
    noniid_partition(X_train, y_train, NUM_CLIENTS)
    if FED_NONIID else
    iid_partition(X_train, y_train, NUM_CLIENTS)
)

if USE_BOOTSTRAP:
    clients_data = bootstrap_client_data(clients_data)

# Quick check: print label distribution
for i, (Xc, yc) in enumerate(clients_data):
    u, c = np.unique(yc, return_counts=True)
    print(f"Client {i} label counts:", dict(zip(u, c)))

Dataset after label encoding + numeric filter: (2830743, 79)
Final dataset shape: (2827876, 78) Labels shape: (2827876,)
Train: (2262300, 78) Test: (565576,)
Client 0 label counts: {0: 290522, 1: 71446}
Client 1 label counts: {0: 193795, 1: 71263}
Client 2 label counts: {0: 290619, 1: 23856}
Client 3 label counts: {0: 290885, 1: 71083}
Client 4 label counts: {0: 290797, 1: 47424}


In [5]:
def get_clients_data(
    X, y,
    num_clients=5,
    noniid=True,
    use_bootstrap=True,
    bootstrap_ratio=0.8,
    seed=42
):
    """Prepare client datasets for federated training."""
    # Partitioning
    if noniid:
        clients_data = noniid_partition(X, y, num_clients, seed)
    else:
        clients_data = iid_partition(X, y, num_clients, seed)

    print("\n--- Before Bootstrapping ---")
    for i, (Xc, yc) in enumerate(clients_data):
        print(f" Client {i}: {Xc.shape[0]} samples, labels={dict(zip(*np.unique(yc, return_counts=True)))}")

    # Optional Bootstrapping
    if use_bootstrap:
        clients_data = bootstrap_client_data(clients_data, bootstrap_ratio, seed)
        print("\n--- After Bootstrapping ---")
        for i, (Xc, yc) in enumerate(clients_data):
            print(f" Client {i}: {Xc.shape[0]} samples, labels={dict(zip(*np.unique(yc, return_counts=True)))}")

    return clients_data

In [ ]:

from sklearn.ensemble import RandomForestClassifier
# Run federated RF
clients_data = get_clients_data(
    X_train, y_train,
    num_clients=NUM_CLIENTS,
    noniid=True,
    use_bootstrap=True
)

final_preds = None  # To hold results

def train_local_rf(X_train, y_train, n_estimators=50, random_state=42):
    """Train Random Forest locally on client data."""
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        random_state=random_state,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model


def federated_random_forest(clients_data, X_test, y_test):
    client_models = []
    for i, (Xc, yc) in enumerate(clients_data):
        print(f"\n--- Training RF on Client {i} ---")
        model = train_local_rf(Xc, yc)
        client_models.append(model)

    # Collect predictions from each client model
    preds = np.array([m.predict(X_test) for m in client_models])
    
    # Majority vote (0/1 classification)
    final_preds = (preds.sum(axis=0) >= (len(client_models) / 2)).astype(int)

    acc = accuracy_score(y_test, final_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, final_preds, average="binary"
    )
    print("\n=== Federated Random Forest Results ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")

    return final_preds

final_preds = federated_random_forest(clients_data, X_test, y_test)



--- Before Bootstrapping ---
 Client 0: 452460 samples, labels={0: 363411, 1: 89049}
 Client 1: 331323 samples, labels={0: 242274, 1: 89049}
 Client 2: 393094 samples, labels={0: 363411, 1: 29683}
 Client 3: 452460 samples, labels={0: 363411, 1: 89049}
 Client 4: 422777 samples, labels={0: 363411, 1: 59366}

--- After Bootstrapping ---
 Client 0: 361968 samples, labels={0: 290522, 1: 71446}
 Client 1: 265058 samples, labels={0: 193795, 1: 71263}
 Client 2: 314475 samples, labels={0: 290619, 1: 23856}
 Client 3: 361968 samples, labels={0: 290885, 1: 71083}
 Client 4: 338221 samples, labels={0: 290797, 1: 47424}

--- Training RF on Client 0 ---

--- Training RF on Client 1 ---

--- Training RF on Client 2 ---

--- Training RF on Client 3 ---

--- Training RF on Client 4 ---

=== Federated Random Forest Results ===
Accuracy: 0.9987
Precision: 0.9973
Recall: 0.9959
F1-score: 0.9966


In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Train Local Decision Tree
def train_local_dt(X_train, y_train, max_depth=None, random_state=42):
    """Train Decision Tree locally on client data."""
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        random_state=random_state
    )
    model.fit(X_train, y_train)
    return model

# Federated Decision Tree
def federated_decision_tree(clients_data, X_test, y_test):
    client_models = []
    for i, (Xc, yc) in enumerate(clients_data):
        print(f"\n--- Training Decision Tree on Client {i} ---")
        model = train_local_dt(Xc, yc)
        client_models.append(model)

    # Collect predictions from each client model
    preds = np.array([m.predict(X_test) for m in client_models])
    
    # Majority vote (0/1 classification)
    final_preds = (preds.sum(axis=0) >= (len(client_models) / 2)).astype(int)

    acc = accuracy_score(y_test, final_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, final_preds, average="binary"
    )
    print("\n=== Federated Decision Tree Results ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")

    return final_preds


final_preds = federated_decision_tree(clients_data, X_test, y_test)



--- Training Decision Tree on Client 0 ---

--- Training Decision Tree on Client 1 ---

--- Training Decision Tree on Client 2 ---

--- Training Decision Tree on Client 3 ---

--- Training Decision Tree on Client 4 ---

=== Federated Decision Tree Results ===
Accuracy: 0.9991
Precision: 0.9969
Recall: 0.9984
F1-score: 0.9976


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# Sample equal data from each dataset 
sample_size = 1000 // 8  # =125
monday_sample = monday.sample(n=sample_size, random_state=42)
tuesday_sample = tuesday.sample(n=sample_size, random_state=42)
wednesday_sample = wednesday.sample(n=sample_size, random_state=42)
thursday_m_sample = thursday_m.sample(n=sample_size, random_state=42)
thursday_a_sample = thursday_a.sample(n=sample_size, random_state=42)
friday_dos_sample = friday_dos.sample(n=sample_size, random_state=42)
friday_portscan_sample = friday_portscan.sample(n=sample_size, random_state=42)
friday_morning_sample = friday_morning.sample(n=sample_size, random_state=42)

small_df = pd.concat([
    monday_sample, tuesday_sample, wednesday_sample,
    thursday_m_sample, thursday_a_sample,
    friday_dos_sample, friday_portscan_sample, friday_morning_sample
], ignore_index=True)

small_df = small_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Subset shape (before preprocessing):", small_df.shape)

# Preprocessing
def preprocess_pipeline(df):
    # Encode labels
    df = encode_labels(df)

    # Keep only numeric
    df = df.select_dtypes(include=[np.number])

    # Features & Labels
    X = df.drop(' Label', axis=1).values
    y = df[' Label'].values

    # Replace inf with NaN, drop NaNs
    X[np.isinf(X)] = np.nan
    mask = ~np.isnan(X).any(axis=1)
    X, y = X[mask], y[mask]

    # Standardize
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

X_small, y_small = preprocess_pipeline(small_df)
print("Subset after preprocessing:", X_small.shape, y_small.shape)

#  Train/Test Split ---
Xs_train, Xs_test, ys_train, ys_test = train_test_split(
    X_small, y_small, test_size=0.2, stratify=y_small, random_state=42
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# --- Gradient Boosting ---
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
gb_model.fit(Xs_train, ys_train)
ys_pred = gb_model.predict(Xs_test)

acc = accuracy_score(ys_test, ys_pred)
prec, rec, f1, _ = precision_recall_fscore_support(
    ys_test, ys_pred, average="binary"
)

print("\n=== Gradient Boosting Results (1000-sample subset) ===")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

final_preds = ys_pred

Subset shape (before preprocessing): (1000, 79)
Subset after preprocessing: (1000, 78) (1000,)

=== Gradient Boosting Results (1000-sample subset) ===
Accuracy:  0.9900
Precision: 0.9756
Recall:    0.9756
F1-score:  0.9756
